# ICLR

## Instalações 

In [1]:
# !pip install transformers datasets accelerate

In [2]:
# !pip install -U fsspec datasets huggingface_hub

## Imports iniciais

In [3]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset

## Carregar dataset

In [4]:
raw_ds = load_dataset("squad")

small_train = raw_ds["train"].shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

## Pré-processar: concatenar contexto + pergunta + resposta

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(ex):
    prompt = ex["context"] + "\n\nQ: " + ex["question"] + "\nA:"
    # junta prompt + resposta
    full = prompt + " " + ex["answers"]["text"][0]
    return tokenizer(full, truncation=True, max_length=512)

tokenized = small_train.map(preprocess, remove_columns=small_train.column_names)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Criar DataCollator

In [6]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

## Modelo

In [7]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
training_args = TrainingArguments(
    output_dir="./phase1",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=50,
    save_steps=200,
    fp16=torch.cuda.is_available(),
    learning_rate=5e-5,
    report_to=["none"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

In [11]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.367500
100,3.245600
150,3.272200
200,3.209500
250,3.213900


TrainOutput(global_step=250, training_loss=3.2617562866210936, metrics={'train_runtime': 56.5816, 'train_samples_per_second': 17.674, 'train_steps_per_second': 4.418, 'total_flos': 127547255808000.0, 'train_loss': 3.2617562866210936, 'epoch': 1.0})

In [12]:
trainer.save_model("./phase1_checkpoint")